In [ ]:
!pip install benchmarx==0.0.11 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
# Imports
from benchmarx import Benchmark
from benchmarx.custom_optimizer import CustomOptimizer, State
from benchmarx.quadratic_problem import QuadraticProblem
import jax.numpy as jnp
from jax import random, grad

In [ ]:
class MyGradientDescent(CustomOptimizer):
    def __init__(self, x_init, stepsize, problem, tol=1e-6, maxiter=400, label = 'MyGD'):
        params = {
            'x_init': x_init,
            'tol': tol,
            'maxiter': maxiter,
            'stepsize': stepsize
        }
        self.stepsize = stepsize
        self.problem = problem
        self.maxiter = maxiter
        self.tol = tol
        super().__init__(params=params, x_init=x_init, label=label)

    def init_state(self, x_init, *args, **kwargs) -> State:
        return State(
            iter_num=1,
            stepsize=self.stepsize
        )

    def update(self, sol, state: State) -> tuple([jnp.array, State]):
        sol -= state.stepsize * grad(self.problem.f)(sol)
        state.iter_num += 1
        state.stepsize = 1 / (state.iter_num + 1)
        return sol, state

    def stop_criterion(self, sol, state: State) -> bool:
        return jnp.linalg.norm(grad(self.problem.f)(sol))**2 < self.tol


In [ ]:
# Let's create a Quadratic Problem
n = 5
x_init = jnp.array([2.0, 1.0, 3.0, .0, .0])
problem = QuadraticProblem(
    n=n,
    mineig=1,
    maxeig=10
)

# Specify your own solver
my_gd_solver = MyGradientDescent(
    x_init=x_init,
    stepsize=1e-2,
    problem=problem,
    tol=1e-3,
    maxiter=300,
    label='MyGD'
)


benchmark = Benchmark(
    runs=2,
    problem=problem,
    methods=[
        {
            'MY_GRADIENT_DESCENT': my_gd_solver
        },
        {
            'GRADIENT_DESCENT_const_step': {
                'x_init' : x_init,
                'tol': 1e-3,
                'maxiter': 300,
                'stepsize' : 1e-1,
                'acceleration': False,
                'label': 'GD_const'
            }
        }
    ],
    metrics=[
        "nit",
        "x",
        "f",
        "grad"
    ],
)

In [ ]:
result = benchmark.run()

In [ ]:
result.plot(
    metrics=['f', 'x_norm', 'f_gap', 'x_gap', 'grad_norm']
)